# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parameterized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [ ]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

model_package_group_name = f"AbaloneModelPackageGroup-Example"
pipeline_name = f"AbalonePipeline-Example"

### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [3]:
from pipelines.abalone.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [4]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:627175950948:pipeline/abalonepipeline-example',
 'ResponseMetadata': {'RequestId': '3259a115-a2b1-476d-9c33-2bb4fb0aeead',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3259a115-a2b1-476d-9c33-2bb4fb0aeead',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Tue, 07 Jun 2022 04:00:16 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [5]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [6]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:627175950948:pipeline/abalonepipeline-example',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:627175950948:pipeline/abalonepipeline-example/execution/3pktnyi7zspx',
 'PipelineExecutionDisplayName': 'execution-1654574417866',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 6, 7, 4, 0, 17, 792000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 6, 7, 4, 0, 17, 792000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:627175950948:user-profile/d-llacda5cqmap/manish-test',
  'UserProfileName': 'manish-test',
  'DomainId': 'd-llacda5cqmap'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:627175950948:user-profile/d-llacda5cqmap/manish-test',
  'UserProfileName': 'manish-test',
  'DomainId': 'd-llacda5cqmap'},
 'ResponseMetadata': {'RequestId': 'cc043027-8d37-452c-b23b-42cb449f79b1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-reques

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [13]:
execution.list_steps()

[{'StepName': 'RegisterAbaloneModel',
  'StartTime': datetime.datetime(2022, 6, 7, 4, 43, 23, 552000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 7, 4, 43, 24, 572000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:627175950948:model-package/abalonemodelpackagegroup-example/3'}}},
 {'StepName': 'CheckMSEAbaloneEvaluation',
  'StartTime': datetime.datetime(2022, 6, 7, 4, 43, 21, 530000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 7, 4, 43, 22, 339000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'EvaluateAbaloneModel',
  'StartTime': datetime.datetime(2022, 6, 7, 4, 38, 39, 630000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 7, 4, 43, 20, 823000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:627175950948:processing-job/pipelines-mup7vf51

### Parameterized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [14]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

### Endpoint Invocation

Once the endpoint is deployed, we can invoke endpoint using sagemaker sdk. In production, we would have front the api with api gateway and lamda. 

In [13]:
import numpy as np
from datetime import datetime, timedelta
import time
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region = boto3.Session().region_name
role = get_execution_role()
sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")




predictions = ""
endpoint_name="manish-project-03-serverless-endpoint"
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
with open("./test/test.csv", "r") as f:
    for row in f:
        print("I am here")
        print(".", end="", flush=True)
        payload = row.rstrip("\n")
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
        )
        predictions =response["Body"].read().decode("utf-8")
        print(predictions)

predictions



Sending test traffic to the endpoint manish-project-03-serverless-endpoint. 
Please wait...
I am here
.9.275335311889648
I am here
.9.275335311889648
I am here
.9.275335311889648


'9.275335311889648'